In [1]:
import os
import argparse
import codecs
import json
import random as rnd
import numpy as np
from collections import Counter, defaultdict
from itertools import chain, count
from six import string_types
import torch
import torchtext.data
import torchtext.vocab

import table
import table.IO
import opts
from tree import SCode

In [2]:
OLD_WHERE_OPS = ('not', 'between', '=', '>', '<', '>=', '<=', '!=', 'in', 'like', 'is', 'exists')
NEW_WHERE_OPS = ('=','>','<','>=','<=','!=','like','not in','in','between','is')
NEW_WHERE_DICT = {
    '=': 0,
    '>': 1,
    '<': 2,
    '>=': 3,
    '<=': 4,
    '!=': 5,
    'like': 6,
    'not in': 7,
    'in': 8,
    'between': 9,
    'is':10
}
# SQL_OPS = ('none','intersect', 'union', 'except')
SQL_OPS = {
    'none': 0,
    'intersect': 1,
    'union': 2,
    'except': 3
}
KW_DICT = {
    'where': 0,
    'groupBy': 1,
    'orderBy': 2
}
ORDER_OPS = {
    'desc': 0,
    'asc': 1}
AGG_OPS = ('none','max', 'min', 'count', 'sum', 'avg')

COND_OPS = {
    'and':0,
    'or':1
}

TRAIN_COMPONENTS = ('multi_sql','keyword','col','op','agg','root_tem','des_asc','having','andor','value')
COMPONENTS_DICT = {
    'multi_sql':0,
    'keyword':1,
    'col':2,
    'op':3,
    'agg':4,
    'root_tem':5,
    'des_asc':6,
    'having':7,
    'andor':8,
    'value':9
}

In [3]:
def convert_to_op_index(is_not,op):
    op = OLD_WHERE_OPS[op]
    if is_not and op == "in":
        return 7
    try:
        return NEW_WHERE_DICT[op]
    except:
        print("Unsupport op: {}".format(op)) # TODO: check ! =
        return -1

def index_to_column_name(index, table):
    column_name = table["column_names"][index][1]
    table_index = table["column_names"][index][0]
    table_name = table["table_names"][table_index]
    return table_name, column_name, index


def get_label_cols(with_join,fk_dict,labels):
    # list(set([l[1][i][0][2] for i in range(min(len(l[1]), 3))]))
    cols = set()
    ret = []
    for i in range(len(labels)):
        cols.add(labels[i][0][2]) # still col index
        if len(cols) > 3:
            break
    for col in cols:
        # ret.append([col])
        if with_join and len(fk_dict[col]) > 0:
            ret.append([col]+fk_dict[col])
        else:
            ret.append(col)
    return ret


# history added
class MultiSqlPredictor:
    def __init__(self, question, sql, history):
        self.sql = sql
        self.question = question
        self.history = history
        self.keywords = ('intersect', 'except', 'union')

    def generate_output(self):
        for key in self.sql:
            if key in self.keywords and self.sql[key]:
                return self.history + ['root'], key, self.sql[key]
        return self.history + ['root'], 'none', self.sql


class KeyWordPredictor:
    def __init__(self, question, sql, history):
        self.sql = sql
        self.question = question
        self.history = history
        self.keywords = ('select', 'where', 'groupBy', 'orderBy', 'limit', 'having')

    def generate_output(self):
        sql_keywords = []
        for key in self.sql:
            if key in self.keywords and self.sql[key]: # included other keywords
                sql_keywords.append(key)
        return self.history, [len(sql_keywords), sql_keywords], self.sql


# history added
class ColPredictor:
    def __init__(self, question, sql, table, history, kw=None):
        self.sql = sql
        self.question = question
        self.history = history
        self.table = table
        self.keywords = ('select', 'where', 'groupBy', 'orderBy', 'having')
        self.kw = kw

    def generate_output(self):
        ret = []
        candidate_keys = self.sql.keys()
        if self.kw:
            candidate_keys = [self.kw]
        for key in candidate_keys:
            if key in self.keywords and self.sql[key]:
                cols = []
                sqls = []
                if key == 'groupBy':
                    sql_cols = self.sql[key]
                    for col in sql_cols:
                        cols.append((index_to_column_name(col[1], self.table), col[2]))
                        sqls.append(col) # col_unit1
                elif key == 'orderBy':
                    sql_cols = self.sql[key][1]
                    for col in sql_cols: # only contain col_unit1 in val_unit: (unit_op, col_unit1, col_unit2)
                        cols.append((index_to_column_name(col[1][1], self.table), col[1][2]))
                        sqls.append(col) # val_unit1
                elif key == 'select':
                    sql_cols = self.sql[key][1]
                    for col in sql_cols:  # only contain col_unit1 in val_unit
                        cols.append((index_to_column_name(col[1][1][1], self.table), col[1][1][2]))
                        sqls.append(col) # (agg_id, val_unit)
                elif key == 'where' or key == 'having':
                    sql_cols = self.sql[key]
                    for col in sql_cols: # TODO: check this one!
                        if not isinstance(col, list):
                            continue
                        try: # col_id of col_unit of val_unit of cond_unit of condition
                            cols.append((index_to_column_name(col[2][1][1], self.table), col[2][1][2]))
                        except:
                            print("Key:{} Col:{} Question:{}".format(key, col, self.question))
                        sqls.append(col) # cond_unit
                ret.append((
                    self.history + [key], (len(cols), cols), sqls
                ))
        return ret
        # ret.append(history+[key],)


class OpPredictor:
    def __init__(self, question, sql, history):
        self.sql = sql # check sql is cond_unit
        self.question = question
        self.history = history # history not change
        # self.keywords = ('select', 'where', 'groupBy', 'orderBy', 'having')

    def generate_output(self): # sql3: val_unit, sql4: val1
        return self.history, convert_to_op_index(self.sql[0],self.sql[1]), (self.sql[3], self.sql[4])


class AggPredictor:
    def __init__(self, question, sql, history,kw=None):
        self.sql = sql
        self.question = question
        self.history = history
        self.kw = kw
    def generate_output(self):
        label = -1
        if self.kw:
            key = self.kw
        else:
            key = self.history[-2]
        if key == 'select':
            label = self.sql[0] # check sql: (agg_id, val_unit)
        elif key == 'orderBy':
            label = self.sql[1][0] # check sql: val_unit1
        elif key == 'having':
            label = self.sql[2][1][0] # check sql: cond_unit
        else: # ADDED
            print("\n Unexpected pre-agg key: ", key)
            exit()
        return self.history, label

# TODO: check why not RootTemPredictor

# class RootTemPredictor:
#     def __init__(self, question, sql):
#         self.sql = sql
#         self.question = question
#         self.keywords = ('intersect', 'except', 'union')
#
#     def generate_output(self):
#         for key in self.sql:
#             if key in self.keywords:
#                 return ['ROOT'], key, self.sql[key]
#         return ['ROOT'], 'none', self.sql


# history added orderBy only one col and agg! TODO: CHECK multiple orderBy columns
class DesAscPredictor:
    def __init__(self, question, sql, table, history):
        self.sql = sql
        self.question = question
        self.history = history
        self.table = table

    def generate_output(self):
        for key in self.sql: # check sql: whole sql
            if key == "orderBy" and self.sql[key]:
                # self.history.append(key)
                try:
                    col = self.sql[key][1][0][1][1] # w
                except:
                    print("question:{} sql:{}".format(self.question, self.sql))
                # self.history.append(index_to_column_name(col, self.table))
                # self.history.append(self.sql[key][1][0][1][0])
                if self.sql[key][0] == "asc" and self.sql["limit"]: # TODO: get limit value and labels
                    label = 0
                elif self.sql[key][0] == "asc" and not self.sql["limit"]:
                    label = 1
                elif self.sql[key][0] == "desc" and self.sql["limit"]:
                    label = 2
                else:
                    label = 3                                           # agg_id in col_unit of val_unit in orderBy
                return self.history+[index_to_column_name(col, self.table), self.sql[key][1][0][1][0]], label


class AndOrPredictor:
    def __init__(self, question, sql, table, history):
        self.sql = sql
        self.question = question
        self.history = history
        self.table = table

    def generate_output(self):
        if 'where' in self.sql and self.sql['where'] and len(self.sql['where']) > 1:
            return self.history, COND_OPS[self.sql['where'][1]]
        return self.history,-1
    
    
def get_table_dict(table_data_path):
    data = json.load(open(table_data_path))
    table = dict()
    for item in data:
        table[item["db_id"]] = item
    return table

In [4]:
def parse_data_full_history(question_tokens, sql, table, history):
    table_schema = [
        table["table_names"],
        table["column_names"],
        table["column_types"]
    ]
    full_labels = []
    masks = [COMPONENTS_DICT['multi_sql']]
    stack = [("root", sql)]
    with_join = False
    fk_dict = defaultdict(list)
    for fk in table["foreign_keys"]:
        fk_dict[fk[0]].append(fk[1])
        fk_dict[fk[1]].append(fk[0])
    while len(stack) > 0:
        node = stack.pop()
        if node[0] == "root":
            if len(masks) > 1:
                masks.append(COMPONENTS_DICT['root_tem'])
                full_labels.append(-1)
            history, label, sql = MultiSqlPredictor(question_tokens, node[1], history).generate_output()
            full_labels.append(SQL_OPS[label])
            history.append(label)
            if label == "none":
                stack.append((label, sql))
                masks.append(COMPONENTS_DICT['keyword'])
            else:
                node[1][label] = None
                stack.append((label, node[1], sql)) # TODO: double check
                masks.append(COMPONENTS_DICT['multi_sql'])
                # if label != "none":
                # stack.append(("none",node[1]))
        elif node[0] in ('intersect', 'except', 'union'):
            stack.append(("root", node[1]))
            stack.append(("root", node[2]))
        elif node[0] == "none":
            with_join = len(node[1]["from"]["table_units"]) > 1
            history, label, sql = KeyWordPredictor(question_tokens, node[1], history).generate_output()
            # full_labels.append(label)
            # [len(sql_keywords), sql_keywords]

            label_idxs = []
            for item in label[1]:
                if item in KW_DICT:
                    label_idxs.append(KW_DICT[item])
            label_idxs.sort()
            full_labels.append(label_idxs)

            if "orderBy" in label[1]:
                stack.append(("orderBy", node[1]))
            if "groupBy" in label[1]:
                has_having = "having" in label[1]

                stack.append(("groupBy", node[1], has_having))
            if "where" in label[1]:
                stack.append(("where", node[1]))
            if "select" in label[1]:
                stack.append(("select", node[1]))

        elif node[0] in ("select", "having", "orderBy"):
            # if node[0] != "orderBy":
            history.append(node[0])
            masks.append(COMPONENTS_DICT['col'])

            col_ret = ColPredictor(question_tokens, node[1], table, history, node[0]).generate_output()
            agg_col_dict = dict()
            op_col_dict = dict()
            if len(col_ret) > 1:
                print("\nWarning: why return more than one col_ret!")
                exit()
            # history + [key], (len(cols), cols), sqls
            for h, l, s in col_ret:
                if l[0] == 0:
                    print("\nWarning: predicted 0 columns!")
                    exit()

                full_labels.append(get_label_cols(with_join, fk_dict, l[1]))
                for col, sql_item in zip(l[1], s):
                    key = "{}{}{}".format(col[0][0], col[0][1], col[0][2]) #table_name, column_name, index
                    # sql_item: (agg_id, val_unit)/select, val_unit1/orderBy, col_unit1/groupBy, cond_unit/where/having
                    if key not in agg_col_dict:
                        agg_col_dict[key] = [(sql_item, col[0])]
                    else:
                        agg_col_dict[key].append((sql_item, col[0])) # for the same col with multiple agg
                    if key not in op_col_dict:
                        op_col_dict[key] = [(sql_item, col[0])]
                    else:
                        op_col_dict[key].append((sql_item, col[0]))
                for key in agg_col_dict:
                    stack.append(("col", node[0], agg_col_dict[key], op_col_dict[key]))
        elif node[0] == "col":
            # full_labels.append(node[2][-1])
            history.append(node[2][0][1])
            if node[1] == "where":
                # stack.append(("value", node[2], "where"))
                stack.append(("op", node[2], "where"))
                # masks.append(COMPONENTS_DICT['op'])
            elif node[1] != "groupBy":
                labels = []
                for sql_item, col in node[2]:
                    _, label = AggPredictor(question_tokens, sql_item, history, node[1]).generate_output()
                    if label - 1 >= 0:
                        labels.append(label - 1) # TODO: check why -1

                # print(node[2][0][1][2])
                masks.append(COMPONENTS_DICT['agg'])
                full_labels.append(labels[:min(len(labels), 3)])

                if node[1] == "having":
                    # stack.append(("value", node[2], "having"))
                    stack.append(("op", node[2], "having"))
                if node[1] == "orderBy":
                    stack.append(("des_asc", sql))

                if len(labels) > 0:
                    history.append(AGG_OPS[labels[0] + 1])

        elif node[0] == "des_asc":
            orderby_ret = DesAscPredictor(question_tokens, node[1], table, history).generate_output()

            if not orderby_ret:
                continue
            masks.append(COMPONENTS_DICT['des_asc'])
            # print(node[1])
            history.append(orderby_ret[1])
            full_labels.append(orderby_ret[1])
            if len(stack) > 0:
                masks.append(-1)
        elif node[0] == "value":
            masks.append([COMPONENTS_DICT['value'],COMPONENTS_DICT['root_tem']])
            val1 = node[1][3]
            val2 = node[1][4]
            if val2:
                if len(stack) > 0:
                    masks.append(-1)
                full_labels.append([1,[val1,val2]])
                history.append([val1,val2])
            else:
                if len(stack) > 0:
                    masks.append(-1)
                full_labels.append([1,[val1]])
                history.append([val1])

        elif node[0] == "op":
            labels = []

            for sql_item, col in node[1]:
                _, label, s = OpPredictor(question_tokens, sql_item, history).generate_output()
                if label != -1:
                    labels.append(label)
                    history.append(NEW_WHERE_OPS[label])

                # masks.append(COMPONENTS_DICT['root_tem'])
                if isinstance(s[0], dict):
                    stack.append(("root", s[0]))
                    masks.append(COMPONENTS_DICT['root_tem'])
                    full_labels.append([0,[]])
                else:
                    stack.append(("value",sql_item))

            if len(labels) > 2:
                print(question_tokens)

            masks.append(COMPONENTS_DICT['op'])
            full_labels.append(labels)
        elif node[0] == "where":
            history.append(node[0])
            hist, andor_label = AndOrPredictor(question_tokens, node[1], table, history).generate_output()

            col_ret = ColPredictor(question_tokens, node[1], table, history, "where").generate_output()
            masks.append(COMPONENTS_DICT['col'])
            op_col_dict = dict()
            for h, l, s in col_ret:
                if l[0] == 0:
                    print("Warning: predicted 0 columns!")
                    continue

                label = get_label_cols(with_join, fk_dict, l[1])
                if len(label) > 1:
                    full_labels.append([label,[andor_label]])
                    masks.append([COMPONENTS_DICT['col'],COMPONENTS_DICT['andor']])
                else:
                    full_labels.append([label,[]])
                    masks.append([COMPONENTS_DICT['col'], COMPONENTS_DICT['andor']])
                # full_labels.append()
                for col, sql_item in zip(l[1], s):
                    key = "{}{}{}".format(col[0][0], col[0][1], col[0][2])
                    if key not in op_col_dict:
                        op_col_dict[key] = [(sql_item, col[0])]
                    else:
                        op_col_dict[key].append((sql_item, col[0]))
                for key in op_col_dict:
                    stack.append(("col", "where", op_col_dict[key]))
        elif node[0] == "groupBy":
            history.append(node[0])
            col_ret = ColPredictor(question_tokens, node[1], table, history, node[0]).generate_output()
            masks.append(COMPONENTS_DICT['col'])
            # agg_col_dict = dict()
            for h, l, s in col_ret:
                if l[0] == 0:
                    print("Warning: predicted 0 columns!")
                    continue

                full_labels.append(get_label_cols(with_join, fk_dict, l[1]))
                if node[2]:
                    stack.append(("having", node[1]))
                    full_labels.append(1)
                    masks.append(COMPONENTS_DICT['having'])
                else:
                    full_labels.append(0)
                    masks.append(COMPONENTS_DICT['having'])

    return history,full_labels,masks


In [20]:
def replace_value(conditions,nl,mp):
    for cond in conditions:
        for i,value in enumerate(cond):
            if i < 3:
                continue
            if not value or isinstance(value,dict):
                continue
            old_value = value
            if isinstance(value,str):
                if value[0] in ('\'','\"'):
                    value = value[1:-1]
                value = value.split()
            else:
                value = [value]
            try:
                new_val = 'VALUE_{}'.format(len(mp))
                cond[i] = new_val
                mp.append(old_value)
                if isinstance(value[0],str):
                    idx = nl.index(value[0])
                else:
                    idx = -1
                    for i in range(len(nl)):
                        if nl[i].isdigit() and (float(nl[i]) == value[0]):
                            idx = i
                            break
                    if idx == -1:
                        # print(old_value)
                        # print(nl)
                        continue
                nl = nl[:idx] + [new_val] + nl[idx+len(value):]
            except Exception:
                # print(old_value)
                # print(nl)
                continue
    return conditions,nl

def replace_nl(sql,nl):
    mp = []
    sql["where"],nl = replace_value(sql["where"],nl,mp)
    sql["having"],nl = replace_value(sql["having"],nl,mp)
    d = {}
    for i,val in enumerate(mp):
        d["VALUE_{}".format(i)] = val
    return d,sql,nl

def parse_data_new_format(data, table_dict):
    dataset = []
    for item in data:
        mp,sql,nl = replace_nl(item["sql"],item["question_toks"])
        history,labels,masks = parse_data_full_history(nl, sql, table_dict[item["db_id"]], [])
        
#         assert len(history) == len(labels) == len(masks)
        print('\n-------------------------------')
        print('len of hisotry: ', len(history), 'len of labels: ', len(labels), "len of masks: ", len(masks))
        print("query  : ", item['query'])
        print("history: ", history)
        print("masks   : ", masks)
        print("label  : ", labels)
        dataset.append({
            "history":history,
            "label":labels,
            "mask":masks,
            "map":mp,
            "nl":nl
        })

In [21]:
data_path = '../../SyntaxSQL/data/dev.json'
table_data_path = '../../SyntaxSQL/data/tables.json'
data = json.load(open(data_path))
table_dict = get_table_dict(table_data_path)
parse_data_new_format(data, table_dict)


-------------------------------
len of hisotry:  5 len of labels:  4 len of masks:  4
query  :  SELECT count(*) FROM singer
history:  ['root', 'none', 'select', ('singer in concert', '*', 0), 'count']
masks   :  [0, 1, 2, 4]
label  :  [0, [], [0], [2]]

-------------------------------
len of hisotry:  5 len of labels:  4 len of masks:  4
query  :  SELECT count(*) FROM singer
history:  ['root', 'none', 'select', ('singer in concert', '*', 0), 'count']
masks   :  [0, 1, 2, 4]
label  :  [0, [], [0], [2]]

-------------------------------
len of hisotry:  9 len of labels:  9 len of masks:  9
query  :  SELECT name ,  country ,  age FROM singer ORDER BY age DESC
history:  ['root', 'none', 'select', ('singer', 'age', 13), ('singer', 'country', 10), ('singer', 'name', 9), 'orderBy', ('singer', 'age', 13), 3]
masks   :  [0, 1, 2, 4, 4, 4, 2, 4, 6]
label  :  [0, [2], [9, 10, 13], [], [], [], [13], [], 3]

-------------------------------
len of hisotry:  9 len of labels:  9 len of masks:  9
query

len of hisotry:  7 len of labels:  7 len of masks:  7
query  :  SELECT max(Accelerate) ,  Cylinders FROM CARS_DATA GROUP BY Cylinders;
history:  ['root', 'none', 'select', ('cars data', 'cylinders', 18), ('cars data', 'accelerate', 22), 'max', 'groupBy']
masks   :  [0, 1, 2, 4, 4, 2, 7]
label  :  [0, [1], [18, 22], [], [0], [18], 0]

-------------------------------
len of hisotry:  9 len of labels:  9 len of masks:  9
query  :  SELECT Model FROM CAR_NAMES GROUP BY Model ORDER BY count(*) DESC LIMIT 1;
history:  ['root', 'none', 'select', ('car names', 'model', 14), 'groupBy', 'orderBy', ('cars data', '*', 0), 'count', 2]
masks   :  [0, 1, 2, 4, 2, 7, 2, 4, 6]
label  :  [0, [1, 2], [14], [], [14], 0, [0], [2], 2]

-------------------------------
len of hisotry:  9 len of labels:  9 len of masks:  9
query  :  SELECT Model FROM CAR_NAMES GROUP BY Model ORDER BY count(*) DESC LIMIT 1;
history:  ['root', 'none', 'select', ('car names', 'model', 14), 'groupBy', 'orderBy', ('cars data', '*', 

-------------------------------
len of hisotry:  11 len of labels:  8 len of masks:  10
query  :  SELECT count(*) FROM FLIGHTS AS T1 JOIN AIRPORTS AS T2 ON T1.DestAirport  =  T2.AirportCode JOIN AIRPORTS AS T3 ON T1.SourceAirport  =  T3.AirportCode WHERE T2.City  =  "Ashley" AND T3.City  =  "Aberdeen"
history:  ['root', 'none', 'select', ('flights', '*', 0), 'count', 'where', ('airports', 'city', 5), '=', '=', ['VALUE_1'], ['VALUE_0']]
masks   :  [0, 1, 2, 4, 2, [2, 8], 3, [9, 5], -1, [9, 5]]
label  :  [0, [0], [0], [2], [[5], []], [0, 0], [1, ['VALUE_1']], [1, ['VALUE_0']]]

-------------------------------
len of hisotry:  9 len of labels:  7 len of masks:  8
query  :  SELECT count(*) FROM FLIGHTS AS T1 JOIN AIRLINES AS T2 ON T1.Airline  =  T2.uid WHERE T2.Airline = "JetBlue Airways"
history:  ['root', 'none', 'select', ('flights', '*', 0), 'count', 'where', ('airlines', 'airline name', 2), '=', ['VALUE_0']]
masks   :  [0, 1, 2, 4, 2, [2, 8], 3, [9, 5]]
label  :  [0, [0], [0], [2], [[

len of hisotry:  9 len of labels:  8 len of masks:  9
query  :  SELECT version_number ,  template_type_code FROM Templates WHERE version_number  >  5
history:  ['root', 'none', 'select', ('templates', 'template type code', 5), ('templates', 'version number', 4), 'where', ('templates', 'version number', 4), '>', ['VALUE_0']]
masks   :  [0, 1, 2, 4, 4, 2, [2, 8], 3, [9, 5]]
label  :  [0, [0], [4, 5], [], [], [[4], []], [1], [1, ['VALUE_0']]]

-------------------------------
len of hisotry:  9 len of labels:  8 len of masks:  9
query  :  SELECT version_number ,  template_type_code FROM Templates WHERE version_number  >  5
history:  ['root', 'none', 'select', ('templates', 'template type code', 5), ('templates', 'version number', 4), 'where', ('templates', 'version number', 4), '>', ['VALUE_0']]
masks   :  [0, 1, 2, 4, 4, 2, [2, 8], 3, [9, 5]]
label  :  [0, [0], [4, 5], [], [], [[4], []], [1], [1, ['VALUE_0']]]

-------------------------------
len of hisotry:  7 len of labels:  7 len of ma

history:  ['root', 'none', 'select', ('projects', 'name', 4), ('assigned to', '*', 0), 'count', 'where', ('projects', 'hours', 5), '>', ['VALUE_0'], 'groupBy']
masks   :  [0, 1, 2, 4, 4, 2, [2, 8], 3, [9, 5], -1, 2, 7]
label  :  [0, [0, 1], [0, 4], [], [2], [[5], []], [1], [1, ['VALUE_0']], [4], 0]

-------------------------------
len of hisotry:  11 len of labels:  10 len of masks:  12
query  :  SELECT count(*) ,  T1.name FROM projects AS T1 JOIN assignedto AS T2 ON T1.code  =  T2.project WHERE T1.hours  >  300 GROUP BY T1.name
history:  ['root', 'none', 'select', ('projects', 'name', 4), ('assigned to', '*', 0), 'count', 'where', ('projects', 'hours', 5), '>', ['VALUE_0'], 'groupBy']
masks   :  [0, 1, 2, 4, 4, 2, [2, 8], 3, [9, 5], -1, 2, 7]
label  :  [0, [0, 1], [0, 4], [], [2], [[5], []], [1], [1, ['VALUE_0']], [4], 0]

-------------------------------
len of hisotry:  7 len of labels:  7 len of masks:  7
query  :  SELECT count(*) ,  T1.name FROM scientists AS T1 JOIN assignedto AS 

label  :  [0, [0], [10, 12], [], [], [[3], []], [0], [1, ['VALUE_0']]]

-------------------------------
len of hisotry:  9 len of labels:  8 len of masks:  9
query  :  SELECT T2.first_name ,  T2.last_name FROM Addresses AS T1 JOIN Staff AS T2 ON T1.address_id = T2.staff_address_id WHERE T1.city = "Damianfort";
history:  ['root', 'none', 'select', ('staff', 'last name', 12), ('staff', 'first name', 10), 'where', ('addresses', 'city', 3), '=', ['VALUE_0']]
masks   :  [0, 1, 2, 4, 4, 2, [2, 8], 3, [9, 5]]
label  :  [0, [0], [10, 12], [], [], [[3], []], [0], [1, ['VALUE_0']]]

-------------------------------
len of hisotry:  11 len of labels:  10 len of masks:  10
query  :  SELECT T1.city ,  count(*) FROM Addresses AS T1 JOIN Staff AS T2 ON T1.address_id = T2.staff_address_id GROUP BY T1.city ORDER BY count(*) DESC LIMIT 1;
history:  ['root', 'none', 'select', ('lessons', '*', 0), 'count', ('addresses', 'city', 3), 'groupBy', 'orderBy', ('lessons', '*', 0), 'count', 2]
masks   :  [0, 1, 2,

len of hisotry:  10 len of labels:  10 len of masks:  10
query  :  SELECT Nationality FROM people GROUP BY Nationality HAVING COUNT(*)  >=  2
history:  ['root', 'none', 'select', ('people', 'nationality', 8), 'groupBy', 'having', ('people', '*', 0), 'count', '>=', ['VALUE_0']]
masks   :  [0, 1, 2, 4, 2, 7, 2, 4, 3, [9, 5]]
label  :  [0, [1], [8], [], [8], 1, [0], [2], [3], [1, ['VALUE_0']]]

-------------------------------
len of hisotry:  8 len of labels:  8 len of masks:  8
query  :  SELECT Name ,  Birth_Date FROM people ORDER BY Name ASC
history:  ['root', 'none', 'select', ('people', 'birth date', 10), ('people', 'name', 9), 'orderBy', ('people', 'name', 9), 1]
masks   :  [0, 1, 2, 4, 4, 2, 4, 6]
label  :  [0, [2], [9, 10], [], [], [9], [], 1]

-------------------------------
len of hisotry:  8 len of labels:  8 len of masks:  8
query  :  SELECT Name ,  Birth_Date FROM people ORDER BY Name ASC
history:  ['root', 'none', 'select', ('people', 'birth date', 10), ('people', 'name', 9),

query  :  SELECT order_id FROM shipments WHERE shipment_date  =  (SELECT max(shipment_date) FROM shipments)
history:  ['root', 'none', 'select', ('shipments', 'order id', 21), 'where', ('shipments', 'shipment date', 24), '=', 'root', 'none', 'select', ('shipments', 'shipment date', 24), 'max']
masks   :  [0, 1, 2, 4, 2, [2, 8], 5, 3, 5, 1, 2, 4]
label  :  [0, [0], [21], [], [[24], []], [0, []], [0], -1, 0, [], [24], [0]]

-------------------------------
len of hisotry:  7 len of labels:  7 len of masks:  7
query  :  SELECT DISTINCT product_name FROM products ORDER BY product_name
history:  ['root', 'none', 'select', ('products', 'product name', 13), 'orderBy', ('products', 'product name', 13), 1]
masks   :  [0, 1, 2, 4, 2, 4, 6]
label  :  [0, [2], [13], [], [13], [], 1]

-------------------------------
len of hisotry:  7 len of labels:  7 len of masks:  7
query  :  SELECT DISTINCT product_name FROM products ORDER BY product_name
history:  ['root', 'none', 'select', ('products', 'produc

masks   :  [0, 1, 2, 4, 2, [2, 8], 5, 3, 5, 1, 2, 4]
label  :  [0, [0], [3], [4], [[1], []], [0, []], [8], -1, 0, [], [[4, 1]], []]

-------------------------------
len of hisotry:  12 len of labels:  12 len of masks:  12
query  :  SELECT avg(grade) FROM Highschooler WHERE id IN (SELECT T1.student_id FROM Friend AS T1 JOIN Highschooler AS T2 ON T1.student_id  =  T2.id)
history:  ['root', 'none', 'select', ('high schooler', 'grade', 3), 'avg', 'where', ('high schooler', 'id', 1), 'in', 'root', 'none', 'select', ('friend', 'student id', 4)]
masks   :  [0, 1, 2, 4, 2, [2, 8], 5, 3, 5, 1, 2, 4]
label  :  [0, [0], [3], [4], [[1], []], [0, []], [8], -1, 0, [], [[4, 1]], []]

-------------------------------
len of hisotry:  12 len of labels:  12 len of masks:  12
query  :  SELECT min(grade) FROM Highschooler WHERE id NOT IN (SELECT T1.student_id FROM Friend AS T1 JOIN Highschooler AS T2 ON T1.student_id  =  T2.id)
history:  ['root', 'none', 'select', ('high schooler', 'grade', 3), 'min', 'whe

In [2]:
UNK_WORD = '<unk>'
UNK = 0
PAD_WORD = '<blank>'
PAD = 1
BOS_WORD = '<s>'
BOS = 2
EOS_WORD = '</s>'
EOS = 3
SKP_WORD = '<sk>'
SKP = 4
RIG_WORD = '<]>'
RIG = 5
LFT_WORD = '<[>'
LFT = 6
special_token_list = [UNK_WORD, PAD_WORD, BOS_WORD, EOS_WORD, SKP_WORD, RIG_WORD, LFT_WORD]

In [3]:
def get_parent_index(tk_list):
    stack = [0]
    r_list = []
    for i, tk in enumerate(tk_list):
        r_list.append(stack[-1])
        if tk.startswith('('):
            # +1: because the parent of the top level is 0
            stack.append(i+1)
        elif tk ==')':
            stack.pop()
    # for EOS (</s>)
    r_list.append(0)
    return r_list


def get_tgt_mask(lay_skip):
    # 0: use layout encoding vectors; 1: use target word embeddings;
    # with a <s> token at the first position
    return [1] + [1 if tk in (SKP_WORD, RIG_WORD) else 0 for tk in lay_skip]


def get_lay_index(lay_skip):
    # with a <s> token at the first position
    r_list = [0]
    k = 0
    for tk in lay_skip:
        if tk in (SKP_WORD, RIG_WORD):
            r_list.append(0)
        else:
            r_list.append(k)
            k += 1
    return r_list


def get_tgt_loss(line, mask_target_loss):
    r_list = []
    for tk_tgt, tk_lay_skip in zip(line['tgt'], line['lay_skip']):
        if tk_lay_skip in (SKP_WORD, RIG_WORD):
            r_list.append(tk_tgt)
        else:
            if mask_target_loss:
                r_list.append(PAD_WORD)
            else:
                r_list.append(tk_tgt)
    return r_list


def __getstate__(self):
    return dict(self.__dict__, stoi=dict(self.stoi))


def __setstate__(self, state):
    self.__dict__.update(state)
    self.stoi = defaultdict(lambda: 0, self.stoi)


torchtext.vocab.Vocab.__getstate__ = __getstate__
torchtext.vocab.Vocab.__setstate__ = __setstate__


def filter_counter(freqs, min_freq):
    cnt = Counter()
    for k, v in freqs.items():
        if (min_freq is None) or (v >= min_freq):
            cnt[k] = v
    return cnt


def merge_vocabs(vocabs, min_freq=0, vocab_size=None):
    """
    Merge individual vocabularies (assumed to be generated from disjoint
    documents) into a larger vocabulary.

    Args:
        vocabs: `torchtext.vocab.Vocab` vocabularies to be merged
        vocab_size: `int` the final vocabulary size. `None` for no limit.
    Return:
        `torchtext.vocab.Vocab`
    """
    merged = Counter()
    for vocab in vocabs:
        merged += filter_counter(vocab.freqs, min_freq)
    return torchtext.vocab.Vocab(merged,
                                 specials=list(special_token_list),
                                 max_size=vocab_size, min_freq=min_freq)


def join_dicts(*args):
    """
    args: dictionaries with disjoint keys
    returns: a single dictionary that has the union of these keys
    """
    return dict(chain(*[d.items() for d in args]))

def _preprocess_json(js):
    t = SCode((js['token'], js['type']))
    js['lay'] = t.layout(add_skip=False)
    js['lay_skip'] = t.layout(add_skip=True)
    assert len(t.target()) == len(js['lay_skip']), (list(zip(t.target(), js['lay_skip'])), ' '.join(js['tgt']))
    js['tgt'] = t.target()

def read_anno_json(anno_path):
    with codecs.open(anno_path, "r", "utf-8") as corpus_file:
        js_list = [json.loads(line) for line in corpus_file]
        js_list = js_list[:5]
        for js in js_list:
            _preprocess_json(js)
    return js_list

In [11]:
js_list = read_anno_json(test_anno)

In [16]:
def process_data(js_list):
    for js in js_list:
        print("\n"+"-"*50)
        for field in ['src', "token", "type", 'lay', "lay_index", "lay_parent_index",\
                      "copy_to_tgt", "copy_to_ext", "tgt_mask", "tgt", "tgt_copy_ext",\
                      "tgt_parent_index", "tgt_loss"]:
            if field in ('src', 'lay', "token", "type"):
                lines = js[field]
            elif field in ('copy_to_tgt','copy_to_ext'):
                lines = js['src']
            elif field in ('tgt',):
                def _tgt(line):
                    r_list = []
                    for tk_tgt, tk_lay_skip in zip(line['tgt'], line['lay_skip']):
                        if tk_lay_skip in (SKP_WORD, RIG_WORD):
                            r_list.append(tk_tgt)
                        else:
                            r_list.append(PAD_WORD)
                    return r_list
                lines = _tgt(js)
            elif field in ('tgt_copy_ext',):
                def _tgt_copy_ext(line):
                    r_list = []
                    src_set = set(line['src'])
                    for tk_tgt in line['tgt']:
                        if tk_tgt in src_set:
                            r_list.append(tk_tgt)
                        else:
                            r_list.append(UNK_WORD)
                    return r_list
                lines = _tgt_copy_ext(js)
            elif field in ('tgt_loss',):
                lines = get_tgt_loss(js, False)
            elif field in ('tgt_mask',):
                lines = get_tgt_mask(js['lay_skip'])
            elif field in ('lay_index',):
                lines = get_lay_index(js['lay_skip'])
            elif field in ('lay_parent_index',):
                lines = get_parent_index(js['lay'])
            elif field in ('tgt_parent_index',):
                lines = get_parent_index(js['tgt'])
            else:
                raise NotImplementedError

            print(field + ": ", lines)

In [17]:
process_data(js_list)


--------------------------------------------------
src:  ['send', 'a', 'signal', '`', 'signal.SIGUSR1', '`', 'to', 'the', 'current', 'process']
token:  ['os', '.', 'kill', '(', 'os', '.', 'getpid', '(', ')', ',', 'signal', '.', 'SIGUSR1', ')']
type:  ['KEYWORD', 'OP', 'KEYWORD', 'OP', 'KEYWORD', 'OP', 'KEYWORD', 'OP', 'OP', 'OP', 'KEYWORD', 'OP', 'KEYWORD', 'OP']
lay:  ['os', '.', 'kill', '(', 'os', '.', 'getpid', '(', ')', ',', 'signal', '.', 'SIGUSR1', ')']
lay_index:  [0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
lay_parent_index:  [0, 0, 0, 0, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, 0]
copy_to_tgt:  ['send', 'a', 'signal', '`', 'signal.SIGUSR1', '`', 'to', 'the', 'current', 'process']
copy_to_ext:  ['send', 'a', 'signal', '`', 'signal.SIGUSR1', '`', 'to', 'the', 'current', 'process']
tgt_mask:  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tgt:  ['<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>', '<blank>',